In [21]:
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import json
from collections import Counter

In [22]:
# DB接続処理
# Neo4jに接続
uri = "bolt://localhost:7687"
user = "neo4j"
password = "abcd7890"

# ドライバを作成
driver = GraphDatabase.driver(uri, auth=(user, password))

In [23]:
# データ読み込み
molecule_db = pd.read_csv('../datas/molecules.csv', names=['id', 'none', 'pubchem_id', 'common_name', 'flavor_profile'], header=0)
flavor_db = pd.read_csv('../datas/flavor_db.csv', names=['id', 'entry_id', 'alias', 'synonyms', 'scientific_name', 'category', 'molecules'], header=0)

flavor_db.drop(['id'], axis=1, inplace=True)
flavor_db.tail()

,,entry_id,alias,synonyms,scientific_name,category,molecules
935,935.0,0,egg,{'Egg'},Chicken,Animal Product,"{6274, 5311110, 644104, 9609, 18827, 527, 1960..."
936,936.0,979,olive oil,{''},Olea europaea L.,additive,"{6184, 31260, 5281168, 8103}"
937,NaN,980,Baking powder,NaN,NaN,NaN,"{11265, 62465, 644104, 12297, 31242, 527, 4114..."
938,NaN,981,Baking soda,NaN,NaN,NaN,{10340}
939,NaN,982,alum,NaN,NaN,NaN,{24856}


In [24]:
molecule_db.tail(20)

,id,none,pubchem_id,common_name,flavor_profile
1771,1771,1771.0,23676745,Potassium Sorbate,{'odorless'}
1772,1772,1772.0,24832101,"Santalol, alpha- and beta-","{'sandalwood', 'sweet', 'woody', 'deep'}"
1773,1773,1773.0,25021769,"2-Propen-1-one, 3-(4-hydroxyphenyl)-1-phenyl-",{'bitter'}
1774,1774,1774.0,44229138,(RS)-norcoclaurinium,"{'milky', 'sweet', 'fruity'}"
1775,1775,1775.0,46779070,S-Methyl 4-methylpentanethioate,{''}
1776,1776,1776.0,53425122,1-(Ethyltrisulfanyl)propane,"{'onion', 'alliaceous', 'green', 'garlic'}"
1777,1777,1777.0,53472027,D-Isoleucine Methyl Ester Hydrochloride,{''}
1778,1778,1778.0,54670067,l-ascorbic acid,{''}
1779,1779,1779.0,57346909,"4H-Pyrrolo[2,1-d]-1,3,5-dithiazine,tetrahydro-...",{''}
1780,1780,1780.0,57357963,33368-82-0,"{'sulfurous', 'alliaceous'}"


In [25]:
flavor_db.tail(2)

,,entry_id,alias,synonyms,scientific_name,category,molecules
938,NaN,981,Baking soda,NaN,NaN,NaN,{10340}
939,NaN,982,alum,NaN,NaN,NaN,{24856}


In [26]:
# kmeans分析のためにベクトル化したflavor_profileを作成してEntryに保存する
# Make all list of all flavors
all_flavors = molecule_db['flavor_profile'].apply(lambda x: x.replace("'", "").strip('{}').split(', ')).tolist()

# Word2Vecモデルを学習
model = Word2Vec(sentences=all_flavors, vector_size=100, window=5, min_count=1, sg=1)

# フレーバーのベクトルを確認
green_vector = model.wv['green']
print(green_vector)

[-0.11024731  0.08865121  0.00104234  0.05108663  0.02458184 -0.19736837
  0.12629808  0.31930017 -0.17324746  0.08396756 -0.03321015 -0.20425396
  0.00152838  0.03489222 -0.00610784 -0.09907925 -0.01383009 -0.19631614
 -0.11185078 -0.3518713   0.03383446  0.06877746  0.05494044 -0.08592452
 -0.11032649  0.02722137 -0.06767049 -0.07568916 -0.20336595 -0.00994436
  0.16688164 -0.05757504  0.08610387 -0.08225907 -0.15717286  0.14035916
  0.06974806 -0.12797907  0.03121371 -0.22622727  0.01563809 -0.20458741
 -0.07907134  0.06918976  0.1550309  -0.00907886 -0.16905282 -0.05399366
  0.2152504   0.08323916  0.05968301 -0.1051513   0.06531842  0.00069853
 -0.06990708  0.04625301  0.10835875 -0.05259502 -0.20486398  0.05590883
  0.04419304  0.02348708 -0.00402463 -0.11754941 -0.14710538  0.1499636
  0.05159127  0.25151354 -0.2215172   0.27388817 -0.0687238   0.00813129
  0.13377978 -0.04101782  0.1346872   0.06986355  0.05916374 -0.05064672
 -0.12126765 -0.00447821 -0.1270708  -0.05232897 -0.

In [27]:

# Make all list of all molecules

## Molecule ノードが存在するか確認し、存在すれば削除する関数
def initialize_molecules(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (e:Entry)-[r:CONTAINS]->(m:Molecule) DELETE r;")
    tx.run("""
        MATCH (m:Molecule)
        DETACH DELETE m;
    """)        

## Moleculeに値を投入する関数
def insert_molecules(tx, molecule_data):
    # flavor_profileがリスト形式の場合、直接使用
    flavor_str = molecule_data['flavor_profile']
    flavors = flavor_str.replace("'", "").strip('{}').split(', ')
      
    flavor_vector = np.mean([model.wv[flavor] for flavor in flavors if flavor in model.wv], axis=0)
        
    tx.run("""
    CREATE (m:Molecule {
        id: $id,
        pubchem_id: $pubchem_id,
        common_name: $common_name,
        flavor_profile: $flavor_profile,
        flavor_vector: $flavor_vector
    })
    """, 
    id=int(molecule_data['id']), 
    pubchem_id=int(molecule_data['pubchem_id']),
    common_name=molecule_data['common_name'],
    flavor_vector=flavor_vector,
    flavor_profile=flavors)

def append_molecules_index(tx):
    # idに対して一意制約を追加（構文修正済み）
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Molecule) REQUIRE m.id IS UNIQUE")
    
    # pubchem_idにインデックスを作成（構文修正済み）
    tx.run("CREATE INDEX IF NOT EXISTS FOR (m:Molecule) ON (m.pubchem_id)")
    
    # flavor_profileにインデックスを作成（構文修正済み）
    tx.run("CREATE INDEX IF NOT EXISTS FOR (m:Molecule) ON (m.flavor_profile)")
    
def append_default_molecules(tx):
    # Moleculeにflavor vector noneというエントリーを作成する
    tx.run("""
        CREATE (m:Molecule {
            common_name: 'None',
            flavor_vector: [
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            ],
            pubchem_id: 0
        });
        """)

# Moleculesのデータを挿入
with driver.session() as session:
    session.execute_write(initialize_molecules)
    for i, molecule in molecule_db.iterrows():
        session.execute_write(insert_molecules, molecule)
    session.execute_write(append_default_molecules)
    session.execute_write(append_molecules_index)


In [28]:
# Make all list of all entries

## Molecule ノードが存在するか確認し、存在すれば削除する関数
def initialize_entry(tx):
    # Molecule ノードの存在を確認
    result = tx.run("MATCH (m:Entry) RETURN m LIMIT 1")
    if result.single():
        tx.run("MATCH (m:Entry) DETACH DELETE m")
    
## Moleculeに値を投入する関数
def insert_entry(tx, entry_data):
    molecules_str = str(entry_data['molecules'])
    molecules = molecules_str.replace("'", "").strip('{}').split(', ')
    molecules = [s for s in molecules if s.strip()]
	
    synonyms_str = str(entry_data['synonyms'])
    synonyms = synonyms_str.replace("'", "").strip('{}').split(', ')
    search_query =  ' '.join(synonyms) + ' ' + str(entry_data['scientific_name']) + ' ' + str(entry_data['category'])
 
    tx.run("""
    CREATE (e:Entry {
        id: $entry_id,
        name: $alias,
        synonyms: $synonyms,
        scientific_name: $scientific_name,
        category: $category,
        search_query: $search_query
    })
    """, 
    entry_id=int(entry_data['entry_id']),
    alias=entry_data['alias'],
    synonyms=entry_data['synonyms'],
    scientific_name=entry_data['scientific_name'],
    category=entry_data['category'],
    search_query=search_query)
    
    if "munster cheese" in entry_data['alias']:
        print(molecules)
    
    for molecule_id in molecules:
        # NOTE: MoleculeのIDが欠損している場合があるので、その場合にはNoneを指定する
        tx.run("""
            MATCH (e:Entry {id: $entry_id})
            OPTIONAL MATCH (m:Molecule {id: $molecule_id})
            OPTIONAL MATCH (default:Molecule {common_name: 'None'})
            WITH e, COALESCE(m, default) AS molecule
            MERGE (e)-[:CONTAINS]->(molecule)
        """, 
        entry_id=int(entry_data['entry_id']),
        molecule_id=int(molecule_id))
    

def append_entry_index(tx):
    # idに対して一意制約を追加（構文修正済み）
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Entry) REQUIRE m.id IS UNIQUE")
    
    # moleculesにインデックスを作成（構文修正済み）
    tx.run("CREATE INDEX IF NOT EXISTS FOR (m:Entry) ON (m.molecules)")

    # moleculesにインデックスを作成（構文修正済み）
    tx.run("DROP INDEX my_text_index IF EXISTS")
    # 新しいインデックスを作成
    tx.run("CREATE FULLTEXT INDEX my_text_index FOR (n:Entry) ON EACH [n.search_query]")
    
# Moleculesのデータを挿入
with driver.session() as session:
    session.execute_write(initialize_entry)
    for i, molecule in flavor_db.iterrows():
        session.execute_write(insert_entry, molecule)
    session.execute_write(append_entry_index)

['5283329', '8194', '8193', '1031', '1032', '12813', '31252', '31253', '31260', '31265', '6184', '1068', '9261', '13357', '31276', '31284', '93236', '31289', '7749', '70', '5284421', '12367', '7762', '612', '20083', '7795', '7284', '7797', '21108', '7800', '1146', '8314', '126', '560255', '10882', '90246', '135', '650', '521869', '7824', '7341', '6322', '22201', '7353', '8892', '10430', '3776', '7361', '445639', '18635', '8908', '10448', '8914', '7894', '5321950', '10976', '7909', '16617', '7409', '11508', '11005', '6321405', '7937', '7938', '65285', '284', '798', '11552', '12587', '7981', '61743', '3893', '8007', '7501', '338', '8025', '878', '19310', '379', '8063', '16255', '13187', '18827', '6549', '2969', '8091', '8093', '8094', '6560', '6561', '6054', '8103', '6569', '6584', '957', '6590', '8129', '454', '12232', '246728', '8139', '24020', '12756', '8158', '8163', '996', '7654', '998', '12777', '62444', '8174', '8180', '123388']


In [29]:
def agregate_vector(tx):
    # sum
    tx.run("""
        MATCH (e:Entry)-[:CONTAINS]->(m:Molecule)
        WHERE m.flavor_vector IS NOT NULL
        WITH e, COLLECT(m.flavor_vector) AS flavor_vectors
        SET e.flavor_vector_sum = REDUCE(
        sum = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
        flavor_vector IN flavor_vectors | 
        [x IN RANGE(0, SIZE(sum)-1) | sum[x] + flavor_vector[x]]
        )
        RETURN e.name, e.flavor_vector_sum;
    """)
    
    # avg
    tx.run("""
        MATCH (entry:Entry)-[:CONTAINS]->(molecule:Molecule)
        WHERE molecule.flavor_vector IS NOT NULL
        WITH entry, COLLECT(molecule.flavor_vector) AS flavor_vectors, COUNT(molecule) AS molecule_count
        // 合計ベクトルを計算して flavor_vector_sum に保存
        SET entry.flavor_vector_sum = REDUCE(
        sum_vector = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
        flavor_vector IN flavor_vectors | 
        [i IN RANGE(0, SIZE(sum_vector)-1) | sum_vector[i] + flavor_vector[i]]
        )
        // 合計ベクトルを molecule_count で割って平均ベクトルを計算し、 flavor_vector_avg に保存
        SET entry.flavor_vector_avg = [i IN RANGE(0, SIZE(entry.flavor_vector_sum)-1) | entry.flavor_vector_sum[i] / molecule_count]
        RETURN entry.name, entry.flavor_vector_sum, entry.flavor_vector_avg;
    """)
    

# 集計したVectorデータをEntry側二保存する
with driver.session() as session:
    session.execute_write(agregate_vector)

In [30]:
# PCAで次元削減する関数
def reduce_dimensions(vectors):
    if len(vectors) == 0:
        return np.zeros(100)  # Word2Vecのベクトルサイズに応じて調整
    
    # データにNaNや無限大が含まれていないかチェック
    if np.isnan(vectors).any() or np.isinf(vectors).any():
        vectors = np.nan_to_num(vectors)
    
    pca = PCA(n_components=1)
    features =  pca.fit_transform(vectors)
    try:
        print(features.components_)
        print(features.explained_variance_ratio_)
        print(features.explained_variance_)
        return features.flatten()
    except Exception as e:
        return np.zeros(100)


In [32]:
# すべてのEntryを取得してPCAを適用
def append_pca_entries(session):
    entries = session.run("MATCH (e:Entry) RETURN e")
    
    for record in entries:
        entry_node = record["e"]  # ノード全体が含まれている
        entry_id = entry_node["id"]  # ノードのIDを取得

        # Entryに接続しているすべてのMoleculeのflavor_profileを統合する
        molecules = session.run("""
            MATCH (e:Entry {id: $entry_id})-[:CONTAINS]->(m:Molecule) RETURN m
        """, {"entry_id": entry_id})
        
        # 各Moleculeのflavor_profileをカウント
        flavor_counter = Counter()
        for molecule_record in molecules:
            molecule_node = molecule_record["m"]
            flavor_profiles = str(molecule_node["flavor_profile"]).strip("[]' ").split("', '")
            flavor_counter.update(flavor_profiles)
        
        # Word2Vecでベクトル化（フレーバーの出現頻度に応じてベクトルを加重平均）
        vectors = []
        for flavor, count in flavor_counter.items():
            if flavor in model.wv:  # Word2Vecモデル内に存在するか確認
                vector = model.wv[flavor] * count  # フレーバーの出現回数で加重
                vectors.append(vector)
        
        if len(vectors) == 0:
            print(f"flavor_profileが見つかりませんでした for Entry {entry_id}")
            continue
        
        # 複数次元の場合にのみPCAを適用
        if len(vectors) > 1 and len(vectors[0]) > 1:
            # PCAを適用（Entryに関連するすべてのflavor_profileで）
            n_components = min(len(vectors), len(vectors[0])) if vectors else 1
            pca = PCA(n_components=n_components)  # 必要な次元に圧縮
            pca.fit(vectors)
            
            # 主成分の寄与率を取得
            explained_variance_ratio = pca.explained_variance_ratio_

            # 各主成分に対する寄与度を取得
            principal_components = pca.components_
            
            flavor_scores = {}
            for i, component in enumerate(principal_components):
                for j, (flavor, _) in enumerate(flavor_counter.items()):  # flavor_counterのキーを参照
                    if j < len(component):  # ベクトルの長さを超えないようにチェック
                        flavor_scores[flavor] = component[j] * explained_variance_ratio[i]  # 寄与率を反映したスコア
            flavor_scores = dict(sorted(flavor_scores.items(), key=lambda x: x[1], reverse=True))
        else:
            # 1次元の場合はそのままスコアとして保存
            mean_vector = np.mean(vector, axis=0)  # 平均ベクトル
            flavor_scores = {flavor: float(mean_vector) for flavor, vector in zip(flavor_counter.keys(), vectors)}            
            
        json_dump_scores = json.dumps(flavor_scores)
        principal_flavor = max(flavor_scores, key=flavor_scores.get)
        
        # Entryノードにflavor_vector_pcaを保存
        session.run(
            "MATCH (e:Entry {id: $entry_id}) SET e.flavor_scores=$flavor_scores, e.flavor_principal=$principal",
            {"entry_id": entry_id, "flavor_scores": json_dump_scores, "principal": principal_flavor}
        )

# メイン処理実行
with driver.session() as session:
    append_pca_entries(session)


flavor_profileが見つかりませんでした for Entry 87
flavor_profileが見つかりませんでした for Entry 90
flavor_profileが見つかりませんでした for Entry 92
flavor_profileが見つかりませんでした for Entry 93
flavor_profileが見つかりませんでした for Entry 95
flavor_profileが見つかりませんでした for Entry 96
flavor_profileが見つかりませんでした for Entry 97
flavor_profileが見つかりませんでした for Entry 100
flavor_profileが見つかりませんでした for Entry 101
flavor_profileが見つかりませんでした for Entry 106
flavor_profileが見つかりませんでした for Entry 115
flavor_profileが見つかりませんでした for Entry 120
flavor_profileが見つかりませんでした for Entry 126
flavor_profileが見つかりませんでした for Entry 127
flavor_profileが見つかりませんでした for Entry 132
flavor_profileが見つかりませんでした for Entry 142
flavor_profileが見つかりませんでした for Entry 144
flavor_profileが見つかりませんでした for Entry 158
flavor_profileが見つかりませんでした for Entry 163
flavor_profileが見つかりませんでした for Entry 166
flavor_profileが見つかりませんでした for Entry 168
flavor_profileが見つかりませんでした for Entry 169
flavor_profileが見つかりませんでした for Entry 170
flavor_profileが見つかりませんでした for Entry 189
flavor_profileが見つかりませんでした for Entry 197
flavor_

In [189]:
# ドライバをクローズ
driver.close()